In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [ ]:
### Each cell is to play with datam so I can get familiar with the format. This way I can parse the data correctly
### when I add it to be database to then be trained

test = 'https://www.basketball-reference.com/leagues/NBA_2016_per_game.html'
response = get(test)
soup = BeautifulSoup(response.text, 'html.parser')

header_str = "Player	Pos	Age	Tm	G	GS	MP	FG	FGA	FG%	3P	3PA	3P%	2P	2PA	2P%\teFG%\tFT	FTA	FT%	ORB	DRB	TRB	" \
          "AST	STL	BLK	TOV	PF	PTS "

# table headers for each stat
headers = [header.strip() for header in header_str.split('\t')]

table = soup.find_all('td')
table_data = [t.text for t in table] # convert table matrix to list

num_rows = int(len(table_data)/len(headers))
row_length = len(headers)

players = []


def fun(x):
    try:
        return float(x)
    except ValueError:
        return x


for i in range(0, len(table_data), row_length):
    # row for player
    row = table_data[i: i+row_length]
    # convert to dict
    player = {headers[i]:fun(row[i]) for i in range(row_length)}
    players.append(player)
    

# create new headers bc not all labels tell the correct story
labels = headers[:]
labels.remove('AGE')
labels.remove('TM')


pt = pd.DataFrame(players)
test = pt[['Player', 'PTS']]
print(pt)




# find which columns need to be refactored to floats
for column in pt:
    print(column, pt[column].dtype)

print(len(table_data), num_rows, row_length)
print('There are ', len(players), ' in 2015/2016 nba season')
    
    



In [25]:
### cell for scraping games won

response = get('https://www.basketball-reference.com/leagues/NBA_2017_standings.html')
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.find_all('td')
l = [table for table in tables]


# filtering functions
def east_function(element):
    return [] != element.find_all('th', attrs={'aria-label':"Eastern Conference"})

def west_function(element):
    return [] != element.find_all('th', attrs={'aria-label':"Western Conference"})

# isolate for correct table
east = list(filter(east_function, l))[0] 
west = list(filter(west_function, l))[0]


# setup dictionary for team_link -> winrate for season
wr = east.find_all('td', attrs={"data-stat":'win_loss_pct'})
teams = east.find_all('a')
east_data = {teams[i]['href']: float(wr[i].text) for i in range(len(wr))}

wr = west.find_all('td', attrs={"data-stat":'win_loss_pct'})
teams = west.find_all('a')
west_data = {teams[i]['href']: float(wr[i].text) for i in range(len(wr))}

print(east_data)
print(west_data)


{'/teams/BOS/2017.html': 0.646, '/teams/CLE/2017.html': 0.622, '/teams/TOR/2017.html': 0.622, '/teams/WAS/2017.html': 0.598, '/teams/ATL/2017.html': 0.524, '/teams/MIL/2017.html': 0.512, '/teams/IND/2017.html': 0.512, '/teams/CHI/2017.html': 0.5, '/teams/MIA/2017.html': 0.5, '/teams/DET/2017.html': 0.451, '/teams/CHO/2017.html': 0.439, '/teams/NYK/2017.html': 0.378, '/teams/ORL/2017.html': 0.354, '/teams/PHI/2017.html': 0.341, '/teams/BRK/2017.html': 0.244}
{'/teams/GSW/2017.html': 0.817, '/teams/SAS/2017.html': 0.744, '/teams/HOU/2017.html': 0.671, '/teams/LAC/2017.html': 0.622, '/teams/UTA/2017.html': 0.622, '/teams/OKC/2017.html': 0.573, '/teams/MEM/2017.html': 0.524, '/teams/POR/2017.html': 0.5, '/teams/DEN/2017.html': 0.488, '/teams/NOP/2017.html': 0.415, '/teams/DAL/2017.html': 0.402, '/teams/SAC/2017.html': 0.39, '/teams/MIN/2017.html': 0.378, '/teams/LAL/2017.html': 0.317, '/teams/PHO/2017.html': 0.293}
